In [1]:
import scipy.sparse as sp
import numpy as np
import torch
import pandas as pd
import pickle
import dgl
import dgl.nn as dglnn
import numpy as np
from dgl.nn.pytorch import GATConv
import dgl.function as fn
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook
import pickle as pkl
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Data

### Load graph

In [3]:
g = dgl.load_graphs('tlx-data/graph_cs_1')[0][0]
g

Graph(num_nodes={'comb': 207689, 'side': 9968},
      num_edges={('comb', 'effect', 'side'): 4576287, ('side', 'effected_by', 'comb'): 4576287},
      metagraph=[('comb', 'side', 'effect'), ('side', 'comb', 'effected_by')])

### Load CSV data

In [4]:
net_comb_side = pd.read_csv('tlx-data/net_comb_side_1.csv')
net_comb_side.head()

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name,combo,No_comb,No_side,S1_No,S2_No
0,CID000002173,CID000003345,C0004144,atelectasis,"CID000002173,CID000003345",27274.0,1793.0,234.0,13.0
1,CID000002173,CID000003345,C0004604,back ache,"CID000002173,CID000003345",27274.0,7809.0,234.0,13.0
2,CID000002173,CID000003345,C0034063,lung edema,"CID000002173,CID000003345",27274.0,6239.0,234.0,13.0
3,CID000002173,CID000003345,C0085631,agitated,"CID000002173,CID000003345",27274.0,4469.0,234.0,13.0
4,CID000002173,CID000003345,C0013384,abnormal movements,"CID000002173,CID000003345",27274.0,7480.0,234.0,13.0


In [5]:
all_sides = list(set(net_comb_side['No_side']))
len(all_sides)

963

In [6]:
min(net_comb_side.No_comb), max(net_comb_side.No_comb), min(net_comb_side.No_side), max(net_comb_side.No_side)

(0.0, 207688.0, 13.0, 9967.0)

In [7]:
node_mapping = pd.read_csv('tlx-data/all_comb_no.csv')
node_mapping.head()

,Unnamed: 0,No
0,"CID000003793,CID000004543",0
1,"CID000002405,CID000004543",1
2,"CID000002405,CID000003793",2
3,"CID000003403,CID000004543",3
4,"CID000003403,CID000003793",4


### Load embeddings

In [8]:
with open('embeddings_ddggs.pkl', 'rb') as fin:
    embeddings = pkl.load(fin)
    
embeddings = {k: v.detach().numpy() for k, v in embeddings.items()}
embeddings

{'drug': array([[-0.36998725, -0.5173515 ,  1.1651148 , ..., -0.5398607 ,
         -1.7846273 , -0.6467879 ],
        [-1.4972149 , -0.48955277,  1.139802  , ...,  0.82100755,
          0.44990215, -1.1496515 ],
        [-0.50894064, -2.0078094 ,  0.59796005, ...,  1.2548794 ,
         -0.37101254, -0.62582314],
        ...,
        [ 0.052891  , -0.32411927, -0.8345813 , ...,  0.02072144,
         -1.1171842 , -1.5496027 ],
        [-1.1898991 ,  0.5039022 , -1.2395205 , ..., -0.4846142 ,
          0.4863882 , -0.897248  ],
        [-0.86853784, -1.4442959 , -1.0452226 , ...,  0.74759537,
         -0.57994866,  1.5941488 ]], dtype=float32),
 'gene': array([[-0.10191067,  0.55739   , -0.91867495, ...,  0.1090597 ,
          0.7077946 ,  1.8194321 ],
        [-0.33868337, -1.5361496 ,  0.10764695, ..., -1.0677509 ,
          0.74651146,  0.27533394],
        [-1.1279132 , -0.77477705, -1.3293167 , ..., -0.2704512 ,
         -0.21833676,  0.4653895 ],
        ...,
        [ 0.65394497,  

In [9]:
embeddings['drug'].sum(axis=1).max(), embeddings['drug'].sum(axis=1).min()

(46.62273, -37.770714)

### comb-nodes mapping

In [10]:
# data_mapping = {row['No_comb']: f"{row['S1_No']},{row['S2_No']}" for idx, row in tqdm_notebook(net_comb_side.iterrows(), total=len(net_comb_side))}


In [11]:
# with open('node_mapping.pkl', 'wb') as fout:
#     pkl.dump(data_mapping, fout)

In [12]:
feature_dim = 128
h_dim = 128
comb_features = np.random.randn(207689, feature_dim)
side_features = np.random.randn(9968, feature_dim)
comb_features.shape, side_features.shape

((207689, 128), (9968, 128))

In [13]:
with open('node_mapping.pkl', 'rb') as fin:
    data_mapping = pkl.load(fin)

side_features = embeddings['side'][:-1]

for k, v in tqdm_notebook(data_mapping.items(), total=len(data_mapping)):
    v0, v1 = [int(float(i)) for i in v.split(',')]
    comb_features[int(k)] = embeddings['drug'][v0] + embeddings['drug'][v1]


  0%|          | 0/63472 [00:00<?, ?it/s]

In [14]:
g.nodes['comb'].data['h'] = torch.FloatTensor(comb_features)
g.nodes['side'].data['h'] = torch.FloatTensor(side_features)

In [15]:
# train_sample = torch.zeros(len(comb_features), dtype=torch.bool).bernoulli(0.8)
# g.nodes['comb'].data['train_mask'] = train_sample
# g.nodes['comb'].data['test_mask'] = ~train_sample

## Training and evaluation

### Build model

In [16]:
class StochasticTwoLayerRGCN(nn.Module):
    def __init__(self, in_feat, hidden_feat, out_feat, rel_names, dropout=0.):
        super().__init__()
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.conv1 = dglnn.HeteroGraphConv({
                rel : dglnn.GraphConv(in_feat, hidden_feat, norm='right', weight=True)
                for rel in rel_names
            })
        self.conv2 = dglnn.HeteroGraphConv({
                rel : dglnn.GraphConv(hidden_feat, out_feat, norm='right', weight=True)
                for rel in rel_names
            })

    def forward(self, blocks, x):
        x = self.conv1(blocks[0], x)
        x = {k: self.dropout1(v) for k, v in x.items()}
        x = self.conv2(blocks[1], x)
        x = {k: self.dropout2(v) for k, v in x.items()}
        return x
       
        
class ScorePredictor(nn.Module):
    
    def forward(self, edge_subgraph, h):
        with edge_subgraph.local_scope():
            edge_subgraph.ndata['h'] = h
            
            for etype in edge_subgraph.canonical_etypes:
                edge_subgraph.apply_edges(
                    dgl.function.u_dot_v('h', 'h', 'score'), 
#                     self.apply_edges, 
                    etype=etype
                )
            return edge_subgraph.edata['score']

        
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, etypes, dropout=0.):
        super().__init__()
#         self.embed = torch.nn.Parameter(torch.Tensor(source_dim, embed_dim))
        self.rgcn = StochasticTwoLayerRGCN(
            in_features, hidden_features, out_features, etypes, dropout=dropout)
        self.pred = ScorePredictor()

    def forward(self, graph, blocks, x):
        x = self.rgcn(blocks, x)
        score = self.pred(graph, x)
        score = {etype: torch.sigmoid(value) for etype, value in score.items()}
        return score


In [29]:
def compute_loss(pos_score, neg_score, margin=0):
    # 损失
    loss = []
    for etype, d in pos_score.items():
        n_edges_pos = pos_score[etype].shape[0]
        n_edges_neg = neg_score[etype].shape[0]
        if n_edges_pos == 0 or n_edges_neg == 0: continue
        loss.append((1 - (pos_score[etype].unsqueeze(1) + margin) + (neg_score[etype].view(n_edges_neg, -1) - margin)).clamp(min=0).mean())
#         loss.append((torch.sigmoid(- pos_score[etype].unsqueeze(1)) + torch.sigmoid(neg_score[etype].view(n_edges_neg, -1))).mean())
    
    if len(loss) == 0:
        raise ValueError('error value')
        
    loss = torch.hstack(loss).sum()
    return loss


def auprc_auroc(target_tensor, score_tensor):
    y = target_tensor.detach().cpu().numpy()
    pred = score_tensor.detach().cpu().numpy()
    auroc, auprc = metrics.roc_auc_score(y, pred), metrics.average_precision_score(y, pred)
#     y, xx, _ = metrics.precision_recall_curve(y, pred)
#     auprc = metrics.auc(xx, y)

    return auprc, auroc


from sklearn import metrics
def compute_auprc_auroc(pos_score, neg_score):
    pos_label = torch.ones(pos_score.shape[0])
    neg_label = torch.zeros(neg_score.shape[0])
    
    pred = torch.cat([pos_score, neg_score])
    label = torch.cat([pos_label, neg_label])
    
    metric = auprc_auroc(label, pred)
    auprc, auroc = metric

    return auprc, auroc


from operator import itemgetter
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

### Build dataloader

In [18]:
batch_size = 2048
test_batch_size = 4096

In [19]:
eid_dict = {etype: g.edges(etype=etype, form='eid') for etype in ['effect']}

train_mask = torch.zeros(len(eid_dict['effect']), dtype=torch.bool).bernoulli(0.8)
test_mask = ~train_mask

train_eid_dict = {'effect': eid_dict['effect'][train_mask]}

test_eid_dict = {'effect': eid_dict['effect'][test_mask]}

sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)

In [20]:
# for eid in test_eid_dict['effect']:
#     src, dst = g.find_edges(eid, etype='effect')
#     if dst in all_sides:
#         print(eid, src, dst)
#     break

In [21]:
# 用于存放每个side相关的边，key为side id，value为边集合
test_eid_by_side = {}
for s in tqdm_notebook(all_sides):
    s = int(s)
    part_eids = dgl.in_subgraph(g, {'side':[s]}).edges['effect'].data['_ID']
    part_eids = set(part_eids.tolist()).intersection(test_eid_dict['effect'].tolist())
    test_eid_by_side[s] = list(part_eids)

  0%|          | 0/963 [00:00<?, ?it/s]

In [22]:
test_dataloader_by_side = {}
for s, eids in tqdm_notebook(test_eid_by_side.items()):
    test_dataloader = dgl.dataloading.DistEdgeDataLoader(
        g, {'effect': eids}, sampler,
        negative_sampler=dgl.dataloading.negative_sampler.Uniform(1),
        batch_size=test_batch_size,
        shuffle=False,
        drop_last=False,
#         num_workers=args.num_workers
    )
    
    test_dataloader_by_side[s] = test_dataloader

  0%|          | 0/963 [00:00<?, ?it/s]

* Remove isolated nodes

In [23]:
need_remove = list(set(range(9968)).difference(list(map(int, all_sides))))
need_remove = np.random.choice(need_remove, 9968 - 963 * 1)
g.remove_nodes(torch.tensor(need_remove), ntype='side')
g

Graph(num_nodes={'comb': 207689, 'side': 4279},
      num_edges={('comb', 'effect', 'side'): 4576287, ('side', 'effected_by', 'comb'): 4576287},
      metagraph=[('comb', 'side', 'effect'), ('side', 'comb', 'effected_by')])

In [24]:
# new_g = dgl.metapath_reachable_graph(g, ['similar'])
# ('drug', 'effect', 'side'): 169906, ('drug', 'has', 'gene'): 18632, ('drug', 'similar', 'drug'): 52557

# test_dataloader = dgl.dataloading.DistEdgeDataLoader(
#     g, test_eid_dict, sampler,
#     negative_sampler=dgl.dataloading.negative_sampler.Uniform(1),
#     batch_size=test_batch_size,
#     shuffle=False,
#     drop_last=False,
# #     num_workers=args.num_workers
# )

### Training

In [30]:
from sklearn.model_selection import KFold

dropout=0.4

ids = test_eid_dict['effect'].tolist()
kf = KFold(n_splits=5)
margin = 0.15
models = []

for fold, (train_ids, val_ids) in enumerate(kf.split(ids)):
    print(f'######## fold {fold + 1}/5 ########')
    
    ### Define dataset ###
    train_eid_dict = {'effect': torch.tensor(train_ids)}
    val_eid_dict = {'effect': torch.tensor(val_ids)}
    
    dataloader = dgl.dataloading.DistEdgeDataLoader(
    g, train_eid_dict, sampler,
        negative_sampler=dgl.dataloading.negative_sampler.Uniform(5),
        batch_size=batch_size,
        shuffle=True,
        drop_last=False,
#     num_workers=args.num_workers
    )
    
    val_dataloader = dgl.dataloading.DistEdgeDataLoader(
    g, val_eid_dict, sampler,
        negative_sampler=dgl.dataloading.negative_sampler.Uniform(1),
        batch_size=test_batch_size * 4,
        shuffle=True,
        drop_last=False,
#     num_workers=args.num_workers
    )
    
    ### Define model and optimizer ###
    model = Model(feature_dim, h_dim, h_dim, g.etypes, dropout=dropout)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    lr = 1e-3
    #### Start training ###
    step = 0
    for i in range(1):
        print(f'#### epoch {i + 1}/1 ####')
        bar = tqdm_notebook(dataloader, total=len(train_eid_dict['effect']) // batch_size)

        for input_nodes, positive_graph, negative_graph, blocks in bar:
            step += 1
        #     blocks = [b.to(torch.device('cuda')) for b in blocks]
        #     positive_graph = positive_graph.to(torch.device('cuda'))
        #     negative_graph = negative_graph.to(torch.device('cuda'))
            input_features = blocks[0].srcdata['h']
            pos_score = model(positive_graph, blocks, input_features)
            neg_score = model(negative_graph, blocks, input_features)

            loss = compute_loss(pos_score, neg_score, margin=margin)
            opt.zero_grad()
            loss.backward()
            opt.step()

            if step % 100 == 0:   
                for params in opt.param_groups:   
                    params['lr'] *= 0.95   
                    lr = params['lr']
            loss_val = loss.detach().item()
            
            ### Validation ###
            if step % 10 == 0:
                input_nodes, positive_graph, negative_graph, blocks = next(iter(val_dataloader))
                input_features = blocks[0].srcdata['h']
                pos_score = model(positive_graph, blocks, input_features)
                neg_score = model(negative_graph, blocks, input_features)
                pos_score_ = pos_score[('comb','effect','side')].detach().numpy().reshape(-1)
                neg_score_ = neg_score[('comb','effect','side')].detach().numpy().reshape(-1)

                eid = 0
                predicted = []
                actual = []
                for x in pos_score_:
                    predicted.append((x, eid))
                    actual.append(eid)
                    eid += 1

                for x in neg_score_:
                    predicted.append((x, eid))
                    eid += 1

                labels_all = np.hstack([np.ones(len(pos_score_)), np.zeros(len(neg_score_))])
                predicted = list(zip(*sorted(predicted, reverse=True, key=itemgetter(0))))[1]

                auprc, auroc = compute_auprc_auroc(pos_score[('comb','effect','side')], neg_score[('comb','effect','side')])
                ap = apk(actual, predicted, k=50)
                
                print(pos_score_, np.min(pos_score_), np.max(pos_score_))
                print(neg_score_, np.min(neg_score_), np.max(neg_score_))
                print(f'train loss:{round(loss_val, 4)}, auprc:{round(np.mean(auprc), 4)}, auroc:{round(np.mean(auroc), 4)}, ap@50:{round(np.mean(ap), 4)}')
                print('=' * 80)
            bar.set_description(f'loss:{round(loss_val, 4)}, lr:{round(lr, 4)}')
    models.append(model)


######## fold 1/5 ########
#### epoch 1/3 ####


  0%|          | 0/357 [00:00<?, ?it/s]

[0.99882704 0.98998296 0.9974533  ... 0.99265975 0.997821   0.999193  ] 0.8853118 0.99998105
[0.44558263 0.4320513  0.43734512 ... 0.43817556 0.43860444 0.43571323] 0.38974565 0.9999635
train loss:0.2785, auprc:0.8597, auroc:0.9082, ap@50:0.8612
[1. 1. 1. ... 1. 1. 1.] 0.99984133 1.0
[0.18931751 0.22248712 0.22461393 ... 0.99999964 0.22103335 0.18441582] 0.093644105 1.0
train loss:0.1599, auprc:0.8357, auroc:0.9009, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.99999857 1.0
[0.08516694 0.06405666 0.051587   ... 0.08481565 0.11367358 1.        ] 0.020042239 1.0
train loss:0.158, auprc:0.8167, auroc:0.8878, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.9999999 1.0
[0.05051127 1.         0.08636585 ... 0.05458018 1.         1.        ] 0.0119412225 1.0
train loss:0.1619, auprc:0.8169, auroc:0.8879, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.99999976 1.0
[0.02027635 0.03339294 0.01956942 ... 0.05587348 0.02850492 0.04129013] 0.0060645915 1.0
train loss:0.1522, auprc:0.8164, auroc:0.8875, ap@50:1.0
[1. 1. 1. ... 1. 1

  0%|          | 0/357 [00:00<?, ?it/s]

[0.96703446 0.9925659  0.99832505 ... 0.98408186 0.9929804  0.98608834] 0.89645046 0.9999511
[0.43742174 0.42664376 0.9946213  ... 0.44665062 0.99057966 0.97393143] 0.413033 0.9999474
train loss:0.2944, auprc:0.8514, auroc:0.9076, ap@50:0.5473
[1. 1. 1. ... 1. 1. 1.] 0.9993228 1.0
[0.17051794 0.25554603 0.20370074 ... 1.         0.9999994  0.20472004] 0.114023075 1.0
train loss:0.1607, auprc:0.8422, auroc:0.905, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.99999595 1.0
[1.         0.05916295 1.         ... 0.18419708 0.0606341  0.14164275] 0.02122132 1.0
train loss:0.1587, auprc:0.8154, auroc:0.8868, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.9999999 1.0
[0.03867513 0.02820361 0.04320493 ... 0.04216545 0.02641368 1.        ] 0.010848556 1.0
train loss:0.157, auprc:0.8164, auroc:0.8875, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 1.0 1.0
[1.         0.06655947 0.03504047 ... 0.0465028  0.02321497 0.03949908] 0.00528198 1.0
train loss:0.1545, auprc:0.816, auroc:0.8873, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 1.0 1.0
[0

...

[0.9963973  0.9949439  0.99860865 ... 0.9996836  0.99991286 0.9991283 ] 0.91310054 0.99999595
[0.44983706 0.43650255 0.45576802 ... 0.44231954 0.4171219  0.41865823] 0.39827946 0.9999809
train loss:0.2812, auprc:0.8531, auroc:0.9061, ap@50:0.8213
[1.        1.        1.        ... 1.        1.        0.9999969] 0.9995425 1.0
[0.19857682 0.18950109 0.18603715 ... 0.1972437  1.         0.18857515] 0.0928508 1.0
train loss:0.1564, auprc:0.8361, auroc:0.901, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.9999963 1.0
[0.10360716 0.0512226  1.         ... 0.12327977 0.10581658 0.07929481] 0.019509418 1.0
train loss:0.1584, auprc:0.816, auroc:0.8873, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 1.0 1.0
[0.06384592 0.07476044 1.         ... 0.109712   0.025711   0.05052844] 0.009515527 1.0
train loss:0.1533, auprc:0.8149, auroc:0.8864, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.9999999 1.0
[0.050951   0.04629261 0.06331304 ... 0.03259858 0.03595217 0.01353893] 0.007476638 1.0
train loss:0.1535, auprc:0.8187, auroc:0.8893,

  0%|          | 0/357 [00:00<?, ?it/s]

[0.99784696 0.9959468  0.9967237  ... 0.99722373 0.9995034  0.99899226] 0.9511068 0.9999871
[0.4321373  0.99627256 0.42156076 ... 0.42910984 0.42124343 0.44735307] 0.38926372 0.9999871
train loss:0.275, auprc:0.8518, auroc:0.9051, ap@50:0.7588
[1. 1. 1. ... 1. 1. 1.] 0.99992 1.0
[1.         0.18133372 1.         ... 1.         0.9999999  1.        ] 0.09232221 1.0
train loss:0.1563, auprc:0.8287, auroc:0.8962, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.99999917 1.0
[0.07445122 1.         0.0869287  ... 1.         0.10538402 0.07238764] 0.017544627 1.0
train loss:0.1538, auprc:0.8127, auroc:0.8848, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.9999993 1.0
[0.06173688 0.05569831 0.03019636 ... 0.02392548 0.02465589 1.        ] 0.008243443 1.0
train loss:0.1637, auprc:0.8175, auroc:0.8884, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 1.0 1.0
[0.02702203 0.06275009 0.0970744  ... 0.041433   0.04619627 0.04003453] 0.006302904 1.0
train loss:0.1596, auprc:0.8127, auroc:0.8848, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 1.0 1.0


  0%|          | 0/357 [00:00<?, ?it/s]

[0.9917995  0.99881434 0.9959182  ... 0.99786216 0.9927602  0.99614906] 0.91516936 0.9999616
[0.4438991  0.99136    0.44838995 ... 0.44057742 0.45126438 0.44970644] 0.4122095 0.9999472
train loss:0.2909, auprc:0.8577, auroc:0.9096, ap@50:0.7066
[1. 1. 1. ... 1. 1. 1.] 0.9998311 1.0
[0.20791985 0.2038671  0.19699122 ... 0.18544324 1.         0.22989748] 0.11957052 1.0
train loss:0.157, auprc:0.8345, auroc:0.9001, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 0.99999917 1.0
[0.10450613 0.10418595 0.04298866 ... 1.         0.07429606 1.        ] 0.019758558 1.0
train loss:0.1567, auprc:0.8158, auroc:0.8871, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 1.0 1.0
[0.0502843  0.0271534  0.04016702 ... 0.06806607 0.05482598 0.05283819] 0.010632955 1.0
train loss:0.159, auprc:0.8167, auroc:0.8878, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 1.0 1.0
[0.0412328  0.02662941 0.03638174 ... 0.02354144 1.         0.06309039] 0.006330935 1.0
train loss:0.1537, auprc:0.8175, auroc:0.8884, ap@50:1.0
[1. 1. 1. ... 1. 1. 1.] 1.0 1.0
[0.02

In [32]:
torch.save(model, 'cls_model_ddggs.pt')

### Evaluation

In [33]:
def cv_predict(models, pos_graph, neg_graph, blocks, input_features):
    folds = len(models)
    assert folds >= 1, ValueError('models cannot be empty')
    pos_score = None
    neg_score = None
    for m in models:
        pos = m(pos_graph, blocks, input_features)
        neg = m(neg_graph, blocks, input_features)
        
        pos = pos[('comb','effect','side')].detach().numpy().reshape(-1)
        neg = neg[('comb','effect','side')].detach().numpy().reshape(-1)
        
        if pos_score is None:
            pos_score = pos
            neg_score = neg
        else:
            pos_score += pos
            neg_score += neg
    return pos_score / folds, neg_score / folds

#### For type

In [34]:
#### Start test ###
tp_auprc = []
tp_auroc = []
tp_apk = []
for tp, loader in test_dataloader_by_side.items():
    print(f'current: {tp}')
    
    bar = tqdm_notebook(loader)
    auprcs = []
    aurocs = []
    aps = []
    pos_scores = []
    neg_scores = []
    actuals = []
    predictions = []
    eid = 0
    for input_nodes, positive_graph, negative_graph, blocks in bar:
        step += 1
        input_features = blocks[0].srcdata['h']
#         pos_score = model(positive_graph, blocks, input_features)
#         neg_score = model(negative_graph, blocks, input_features)
#         pos_score_ = pos_score[('comb','effect','side')].detach().numpy().reshape(-1)
#         neg_score_ = neg_score[('comb','effect','side')].detach().numpy().reshape(-1)
        pos_score, neg_score = cv_predict(models, positive_graph, negative_graph, blocks, input_features)
            
        predict = []
        actual = []
        for x in pos_score:
            predict.append((x, eid))
            actual.append(eid)
            eid += 1

        for x in neg_score:
            predict.append((x, eid))
            eid += 1
        
        pos_scores.extend(pos_score.tolist())
        neg_scores.extend(neg_score.tolist())
        predictions.extend(predict)
        actuals.extend(actual)

        predict = list(zip(*sorted(predict, reverse=True, key=itemgetter(0))))[1]

        auprc, auroc = compute_auprc_auroc(torch.tensor(pos_score), torch.tensor(neg_score))
        ap50 = apk(actual, predict, k=50)

#         auprcs.append(auprc)
#         aurocs.append(auroc)
#         aps.append(ap50)
        bar.set_description(f'auprc:{round(np.mean(auprc), 4)}, auroc:{round(np.mean(auroc), 4)}, ap@50:{round(np.mean(ap50), 4)}')


    predictions = list(zip(*sorted(predictions, reverse=True, key=itemgetter(0))))[1]
    auprc, auroc = compute_auprc_auroc(torch.tensor(pos_scores), torch.tensor(neg_scores))
    ap50 = apk(actuals, predictions, k=50)
    tp_auprc.append(auprc)
    tp_auroc.append(auroc)
    tp_apk.append(ap50)
    print(f'For type {tp}, metrics:: auprc:{round(np.mean(auprc), 4)}, auroc:{round(np.mean(auroc), 4)}, ap@50:{round(np.mean(ap50), 4)}')

current: 4096


0it [00:00, ?it/s]

For type 4096, metrics:: auprc:0.8098, auroc:0.8826, ap@50:1.0
current: 4097


0it [00:00, ?it/s]

For type 4097, metrics:: auprc:0.838, auroc:0.9034, ap@50:1.0
current: 6146


...

For type 4086, metrics:: auprc:0.8339, auroc:0.9004, ap@50:1.0
current: 4093


0it [00:00, ?it/s]

For type 4093, metrics:: auprc:0.8268, auroc:0.8952, ap@50:1.0
current: 4094


0it [00:00, ?it/s]

For type 4094, metrics:: auprc:0.8068, auroc:0.8803, ap@50:1.0


#### For batch

In [ ]:
#### Start test ###
auprcs = []
aurocs = []
aps = []

bar = tqdm_notebook(test_dataloader, total=len(test_eid_dict['effect']) // batch_size)

actuals = []
predictions = []
eid = 0
for input_nodes, positive_graph, negative_graph, blocks in bar:
    step += 1
    input_features = blocks[0].srcdata['h']
    pos_score = model(positive_graph, blocks, input_features)
    neg_score = model(negative_graph, blocks, input_features)

    pos_score_ = pos_score[('comb','effect','side')].detach().numpy().reshape(-1)
    neg_score_ = neg_score[('comb','effect','side')].detach().numpy().reshape(-1)

    
    predict = []
    actual = []
    for x in pos_score_:
        predict.append((x, eid))
        actual.append(eid)
        eid += 1

    for x in neg_score_:
        predict.append((x, eid))
        eid += 1
    
    predictions.extend(predict)
    actuals.extend(actual)
    
    predict = list(zip(*sorted(predict, reverse=True, key=itemgetter(0))))[1]

    auprc, auroc = compute_auprc_auroc(pos_score[('comb','effect','side')], neg_score[('comb','effect','side')])
    ap50 = apk(actual, predict, k=50)

    auprcs.append(auprc)
    aurocs.append(auroc)
    aps.append(ap50)
    bar.set_description(f'auprc:{round(np.mean(auprcs), 4)}, auroc:{round(np.mean(aurocs), 4)}, ap@50:{round(np.mean(aps), 4)}')
    

predictions = list(zip(*sorted(predictions, reverse=True, key=itemgetter(0))))[1]
auprc, auroc = compute_auprc_auroc(pos_score[('comb','effect','side')], neg_score[('comb','effect','side')])
ap50 = apk(actual, predict, k=50)

-----

In [ ]:
ddgg_1 loss:0.0979, auprc:0.8757, auroc:0.9728, ap:0.7924

In [26]:
# edges = []
# src_nodes = []
# dst_nodes = []
# ps = []
# for i, p in tqdm_notebook(zip(positive_graph.edges['effect'].data['_ID'], positive_graph.edges['effect'].data['p'])):
#     edges.append(i.item())
#     src_node, dst_node = g_.find_edges(i.item(), etype='effect')
# #     node_pairs.append([src_node.item(), dst_node.item()])
#     src_nodes.append(src_node.item())
#     dst_nodes.append(dst_node.item())
#     ps.append(p.item())
# #     g_.edges[src_node.item(), dst_node.item()].data['p'] = p

# g_.edges[src_nodes, dst_nodes].data['p'] = torch.tensor(ps)

In [27]:
# all_sides = set(net_comb_side['No_side'])

# edges = []
# src_nodes = []
# dst_nodes = []
# side_scores = {}
# for i, p in tqdm_notebook(zip(positive_graph.edges['effect'].data['_ID'], positive_graph.edges['effect'].data['p'])):
#     edges.append(i.item())
#     src_node, dst_node = g_.find_edges(i.item(), etype='effect')
#     src_node = src_node.item()
#     dst_node = dst_node.item()
#     if dst_node  in all_sides:
#         side_scores.setdefault(dst_node, [])
#         side_scores[dst_node].append(p)

